In [1]:
using LinearAlgebra
using Printf

# Example

Do an SCF calculation for the helium-atom ground state using a basis set of two $1s$ STOs with obital exponents $\zeta_1 = 1.45$ and $\zeta_2 = 2.91$.

$\chi_i = 2 \zeta_i^{\frac{3}{2}} \exp [-\zeta_i r] Y_0^0$

In [2]:
ζ = [1.45, 2.91]

2-element Vector{Float64}:
 1.45
 2.91

Overlap integrals $S_{rs}$

In [3]:
S = zeros(2, 2)

for r in 1: 2
    for s in 1: 2
        if r == s
            # S_11, S_22
            S[r, s] = 1
        else
            # S_12, S_21
            S[r, s] = 8 * ζ[1]^(3/2) * ζ[2]^(3/2) / (ζ[1] + ζ[2])^3
        end
    end
end

S

2×2 Matrix{Float64}:
 1.0       0.836608
 0.836608  1.0

$$
\hat{H}^{\mathrm{core}} = -\frac{1}{2}\nabla^2 - \frac{2}{r} = - \frac{1}{2} \nabla^2 - \frac{\zeta}{r} + \frac{\zeta - 2}{r}
$$

In [4]:
H_core = zeros(2, 2)

for r in 1: 2
    for s in 1: 2
        if r == s
            # H_11, H_22
            H_core[r, s] = 1/2 * ζ[r]^2 - 2 * ζ[r]
        else
            # H_12, H_21
            H_core[r, s] = ζ[1]^(3/2) * ζ[2]^(3/2) * (4 * ζ[1] * ζ[2] - 8 * ζ[1] - 8 * ζ[2]) / (ζ[1] + ζ[2])^3
        end
    end
end

H_core

2×2 Matrix{Float64}:
 -1.84875  -1.88258
 -1.88258  -1.58595

Electron-repulsion integrals $\left( rs | tu \right)$

In [5]:
R = zeros(2, 2, 2, 2)

for r in 1: 2
    for s in 1: 2
        for t in 1: 2
            for u in 1:2
                if r == s == t == u
                    # (11|11), (22|22)
                    R[r, s, t, u] = 5/8 * ζ[r]
                elseif r == s && t == u && r != t
                    # (11|22), (22|11)
                    R[r, s, t, u] = (ζ[1]^4 * ζ[2] + 4 * ζ[1]^3 * ζ[2]^2 + ζ[1] * ζ[2]^4 + 4 * ζ[1]^2 * ζ[2]^3) / (ζ[1] + ζ[2])^4
                elseif r + s == t + u
                    # (12|12), (21|12), (12|21), (21|21)
                    R[r, s, t, u] = 20 * ζ[1]^3 * ζ[2]^3 / (ζ[1] + ζ[2])^5
                else
                    if sum([r, s, t, u] .== 1) == 3
                        # (11|12), (11|21), (12|11), (21|11)
                        i = 1
                        j = 2
                    else
                        # (12|22), (22|12), (21|22), (22|21)
                        i = 2
                        j = 1
                    end
                    R[r, s, t, u] = 16 * ζ[i]^(9/2) * ζ[j]^(3/2) / (3 * ζ[i] + ζ[j])^4 * ((12 * ζ[i] + 8 * ζ[j]) / (ζ[i] + ζ[j])^2 + (9 * ζ[i] + ζ[j]) / (2 * ζ[i]^2))
                end
            end
        end
    end
end

R

2×2×2×2 Array{Float64, 4}:
[:, :, 1, 1] =
 0.90625   0.903281
 0.903281  1.18259

[:, :, 2, 1] =
 0.903281  0.953631
 0.953631  1.298

[:, :, 1, 2] =
 0.903281  0.953631
 0.953631  1.298

[:, :, 2, 2] =
 1.18259  1.298
 1.298    1.81875

Expand the spatial orbitals $\phi_i$ as linear combinations of a set of one-electron basis functions $\chi_s$:

$$
\phi_i = \sum_{s = 1}^b c_{si} \chi_s
$$

If $b$ is large enough and the functions $\chi_s$ well chosen, one can represent the MOs with negligible error.

For exmaple, when $b = 2$

$$
\phi_1 = c_{11} \chi_1 + c_{21} \chi_2
$$

Take an initial guess

$$
\frac{c_{11}}{c_{12}} = 2
$$

The normalization condition

$$
\int {\left| \phi_1 \right|}^2 \mathrm{d} \tau = 1
$$

gives for the real coefficients

$$
c_{21} = (1 + k^2 + 2k S_{12})^{-\frac{1}{2}}
$$

where

$$
k = \frac{c_{11}}{c_{21}}
$$

In [6]:
k = 2
C = zeros(2, 2)
C[2, 1] = (1 + k^2 + 2 * k * S[1, 2])^(-1/2)
C[1, 1] = k * C[2, 1]

C

2×2 Matrix{Float64}:
 0.692276  0.0
 0.346138  0.0

Density matrix

$$
P_{tu} = 2 \sum_{j = 1}^{n/2} c_{tj}^* c_{uj} \\
t = 1, 2, \cdots, b \\
u = 1, 2, \cdots, b
$$

In [7]:
P = zeros(2, 2)

for t in 1: 2
    for u in 1: 2
        for j in 1: Int64(2/2)
            P[t, u] += C[t, j] * C[u, j]
        end
        P[t, u] *= 2
    end
end

P

2×2 Matrix{Float64}:
 0.958493  0.479247
 0.479247  0.239623

$$
F_{rs} = H_{rs}^{\text{core}} + \sum_{t = 1}^{b} \sum_{u = 1}^{b} P_{tu}\left[\left(rs|tu\right) - \frac{1}{2}\left(ru|ts\right)\right]
$$

In [8]:
F = zeros(2, 2)

for r in 1: 2
    for s in 1: 2
        F[r, s] = H_core[r, s]
        for t in 1: 2
            for u in 1: 2
                F[r, s] += P[t, u] * (R[r, s, t, u] - 1/2 * R[r, u, t, s])
            end
        end
    end
end

F

2×2 Matrix{Float64}:
 -0.812418  -0.892007
 -0.892007  -0.0695028

Matrix Form of the Roothaan Equations

$$
\sum_{s=1}^b F_{rs}c_{si} = \sum_{s=1}^{b}S_{rs}c_{si}\varepsilon_{i} \quad r = 1, 2, \cdots, b
$$

$$
\mathbf{FC= SC\varepsilon}
$$

$$
\mathbf{F^\prime C^\prime = C^\prime \varepsilon}
$$

$$
\mathbf{A = S^\frac{1}{2}}
$$

$$
\mathbf{F^\prime = A^{\dag} F A}
$$

$$
\mathbf{C = A C^\prime}
$$

$$
\mathbf{P = 2  C_{occ} C_{occ}^\dag}
$$

In [11]:
mutable struct State
    S::Matrix{Float64}      # Overlap Integral
    H_core::Matrix{Float64} # Core Hamilton
    R::Array{Float64, 4}    # Two-electron Integral
    C::Matrix{Float64}      # MO Ceofficient
    P::Matrix{Float64}      # Densitry
    F::Matrix{Float64}      # Fock
    ε::Array{Float64}       # MO Energy
end

function State(; S::Matrix{Float64}, H_core::Matrix{Float64}, R::Array{Float64, 4}, P::Matrix{Float64})
    state = State(
        S,
        H_core,
        R,
        zeros(2, 2),
        P,
        zeros(2, 2),
        zeros(2)
    )
end

State

In [12]:
function update!(state::State)
    # Calculate Fock matrix F using H_core, P, & R
    for r in 1: 2
        for s in 1: 2
            state.F[r, s] = state.H_core[r, s]
            for t in 1: 2
                for u in 1: 2
                    state.F[r, s] += state.P[t, u] * (state.R[r, s, t, u] - 1/2 * state.R[r, u, t, s])
                end
            end
        end
    end

    # calculate the A matrix using the overlap integrals S_rs
    A = state.S^(-1/2)

    # calculate the matrix F′
    F′ = A' * state.F * A

    # sovle F′C′ = C′ε
    state.ε, C′ = eigen(F′)
    
    # calculate C
    state.C = A * C′

    # update P
    state.P = 2 * state.C[:, 1] * state.C[:, 1]'
end

update! (generic function with 1 method)

In [13]:
function energy(state::State)
    E_HF = 0.0
    for r in 1: 2
        for s in 1: 2
            E_HF += state.P[r, s] * (state.F[r, s] + state.H_core[r, s])
        end
    end
    E_HF = E_HF * 0.5
end

energy (generic function with 1 method)

In [20]:
state = State(S=S, H_core=H_core, R=R, P=P)
E_old = energy(state)
E_new = 0.0
ΔE = Inf

while abs(ΔE) > 1e-8
    update!(state)
    E_new = energy(state)
    ΔE = E_new - E_old
    @printf "E_new = %.8f  E_old = %.8f  ΔE = %.5e\n" E_new E_old ΔE
    E_old = E_new
end


E_new = -2.79900671  E_old = -1.97824155  ΔE = -8.20765e-01
E_new = -2.85908995  E_old = -2.79900671  ΔE = -6.00832e-02
E_new = -2.86154402  E_old = -2.85908995  ΔE = -2.45407e-03
E_new = -2.86166339  E_old = -2.86154402  ΔE = -1.19373e-04
E_new = -2.86166924  E_old = -2.86166339  ΔE = -5.85480e-06
E_new = -2.86166953  E_old = -2.86166924  ΔE = -2.87273e-07
E_new = -2.86166955  E_old = -2.86166953  ΔE = -1.40957e-08
E_new = -2.86166955  E_old = -2.86166955  ΔE = -6.91633e-10


Final Energy: $\mathrm{-2.86166955 \; E_h}$